In [ ]:
Q1

In [31]:
import pandas as pd
import numpy as np

In [8]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

In [12]:
import ssl
import pandas as pd
import urllib.request

# Bypass SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

github_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'
df = pd.read_csv(github_url)
df = df.iloc[:300]

## Q1. Getting the embeddings model

In [36]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

In [37]:
answer_llm = df.iloc[0].answer_llm

In [22]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [40]:
embeddings = embedding_model.encode(answer_llm)

In [41]:
embeddings

array([-4.22446579e-01, -2.24855691e-01, -3.24058741e-01, -2.84758866e-01,
        7.25708902e-03,  1.01186641e-01,  1.03716828e-01, -1.89983517e-01,
       -2.80596483e-02,  2.71588504e-01, -1.15337148e-01,  1.14666402e-01,
       -8.49587694e-02,  3.32365334e-01,  5.52724376e-02, -2.22195536e-01,
       -1.42540991e-01,  1.02519184e-01, -1.52333423e-01, -2.02912375e-01,
        1.98422819e-02,  8.38149786e-02, -5.68631887e-01,  2.32843235e-02,
       -1.67292640e-01, -2.39256412e-01, -8.05465057e-02,  2.57083327e-02,
       -8.15466121e-02, -7.39289969e-02, -2.61550128e-01,  1.92576181e-02,
        3.22909296e-01,  1.90357268e-01, -9.34726486e-05, -2.13165566e-01,
        2.88943667e-02, -1.79528147e-02, -5.92762418e-02,  1.99918032e-01,
       -4.75170687e-02,  1.71633765e-01, -2.45915428e-02, -9.38056633e-02,
       -3.57002944e-01,  1.33263484e-01,  1.94045842e-01, -1.18530914e-01,
        4.56915259e-01,  1.47728145e-01,  3.35945159e-01, -1.86959863e-01,
        2.45954141e-01, -

## Q2. Computing the dot product

In [42]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [44]:
embeddings_answer_llm = embedding_model.encode(df['answer_llm'])
embeddings_answer_orig = embedding_model.encode(df['answer_orig'])

In [49]:
len(embeddings_answer_llm[0])

768

In [50]:
dot_products = np.einsum('ij,ij->i', embeddings_answer_llm, embeddings_answer_orig)

In [57]:
np.percentile(dot_products,75)

np.float32(31.674305)

## Q3. Computing the cosine

In [58]:
def v_norm(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [63]:
embeddings_answer_llm_norm = np.array(pd.DataFrame(embeddings_answer_llm).apply(v_norm, axis = 1))
embeddings_answer_orig_norm = np.array(pd.DataFrame(embeddings_answer_orig).apply(v_norm, axis = 1))

In [66]:
cosine_similarity = np.einsum('ij,ij->i', embeddings_answer_llm_norm, embeddings_answer_orig_norm)

In [67]:
np.percentile(cosine_similarity,75)

np.float32(0.8362348)

## Q4. Rouge

In [75]:
df['answer_orig'][10]

'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.'

In [77]:
from rouge import Rouge
rouge_scorer = Rouge()
scores = rouge_scorer.get_scores(df['answer_llm'][10], df['answer_orig'][10])[0]

In [82]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

## Q5. Average rouge score

In [81]:
(scores['rouge-1']['r'] + scores['rouge-2']['r'] + scores['rouge-l']['r']) / 3

0.35490035490035493

## Q6. Average rouge score for all the data points

In [99]:
def get_rouge_score(x,y):
    scores = rouge_scorer.get_scores(x, y)[0]
    # rouge_1 = scores['rouge-1']['f']
    # rouge_2 = scores['rouge-2']['f']
    # rouge_l = scores['rouge-l']['f']
    return scores['rouge-2']['f']

In [100]:
df['rouge_score'] = df.apply(lambda row: get_rouge_score(row['answer_llm'], row['answer_orig']), axis=1)

In [101]:
df['rouge_score'].mean()

np.float64(0.20696501983423318)

In [90]:
y = df['answer_orig'][10]

In [91]:
scores = rouge_scorer.get_scores(x, y)

In [92]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']

TypeError: list indices must be integers or slices, not str

In [93]:
scores

[{'rouge-1': {'r': 0.45454545454545453,
   'p': 0.45454545454545453,
   'f': 0.45454544954545456},
  'rouge-2': {'r': 0.21621621621621623,
   'p': 0.21621621621621623,
   'f': 0.21621621121621637},
  'rouge-l': {'r': 0.3939393939393939,
   'p': 0.3939393939393939,
   'f': 0.393939388939394}}]